# Day 8

In [2]:
import time

## Part 1

In [3]:
with open('input.txt', 'r') as f:
    data = f.read().split('\n')
    
digits = {1:0, 4:0, 7:0, 8:0}
for i in range(len(data)):
    # get rid of everything before the ' | '  character
    possibles = data[i].split(' ')[11:]

    for j in range(len(possibles)):
        if len(possibles[j]) == 2:
            digits[1] += 1
        elif len(possibles[j]) == 4:
            digits[4] += 1
        elif len(possibles[j]) == 3:
            digits[7] += 1
        elif len(possibles[j]) == 7:
            digits[8] += 1
    
print(sum(digits.values()))

476


## Part 2

We need to figure out where goes each segment and also what segments represent each number, then we can decode the final values.

In [4]:
with open('input.txt', 'r') as f:
    valores = f.read().split('\n')
start=time.perf_counter_ns()

# gets ['dbc', 'gfecab', 'afcdg', 'dfebcag', 'bd', 'dgbe', 'bcaeg', 'dcefab', 'ecgadb', 'agcbd', '|', 'acdgb', 'gbcda', 'gdecfba', 'bacge'] as input
def get_combination(data):
    digits = {0:'', 1:'', 2:'', 3:'', 4:'', 5:'', 6:'', 7:'', 8:'', 9:''}
    segments = {'top': '', 'top_left': '', 'top_right': '', 'middle': '', 'bottom_left': '', 'bottom_right': '', 'bottom': '', 'middle_segs': '', 'left_segs': '', 'right_segs': ''}
    
    # First we get the easy digits
    for j in range(len(data)):
        if len(data[j]) == 2 and digits[1] == '':
            digits[1] += data[j]
        elif len(data[j]) == 4 and digits[4] == '':
            digits[4] += data[j]
        elif len(data[j]) == 3 and digits[7] == '':
            digits[7] += data[j]
        elif len(data[j]) == 7 and digits[8] == '':
            digits[8] += data[j]
            
    # We can get the top segment because it's the segment in 7 that are not in 1
    segments['top'] = ''.join([x for x in digits[7] if x not in digits[1]])
    
    # we can get the digit 3 letters because it's the 5 letter word that has all letter from digit 1
    for j in range(len(data)):
        if len(data[j]) == 5 and all(x in data[j] for x in digits[1]):
            digits[3] += data[j]
            break
    
    # top, middle and bottom segments, segd in 3 that are not in 1
    segments['middle_segs'] = [x for x in digits[3] if x not in digits[1]]
    # left segments segs in 8 that are not in 3
    segments['left_segs'] = [x for x in digits[8] if x not in digits[3]]
    # right segments segs in 1
    segments['right_segs'] = [x for x in digits[1]]
    
    # middle segment is taking segs in 3 that are not in 7 and cheking which of those two are in 4
    segments['middle'] = [x for x in digits[3] if x not in digits[7] and x in digits[4]][0]
    
    # 0 is the segments in 8 without the middle segment
    digits[0] = ''.join([x for x in digits[8] if x != segments['middle']])

    # 9 is the one with 6 segments that doesn't have both left_segs
    for j in range(len(data)):
        if len(data[j]) == 6 and any(x not in data[j] for x in segments['left_segs']):
            digits[9] += data[j]
            break
        
    # bottom_left is the segment that is is 8 but not in 9
    segments['bottom_left'] = [x for x in digits[8] if x not in digits[9]][0]
    # top_left is the segment in left_segs that is not bottom_left
    segments['top_left'] = [x for x in segments['left_segs'] if x != segments['bottom_left']][0]
    
    # bottom is the segments in 3, but without the middle, top and right_segs
    segments['bottom'] = ''.join([x for x in digits[3] if x not in segments['middle'] and x not in segments['top'] and x not in segments['right_segs']])
    
    # top_right is taking the digits with 5 letters from data and checking which of those two have a letter right_segs in common, that is the top_right
    for j in range(len(data)):
        five_digits = set([''.join(sorted(x)) for x in data if len(x) == 5])
    five_digits = list(five_digits)
    # now we can check which of the three elements has bottom_left and that's digit 2
    for j in range(len(five_digits)):
        if segments['bottom_left'] in five_digits[j]:
            digits[2] = five_digits[j]
            break
    # now we can check which of the three elements has top_left and that's digit 5
    for j in range(len(five_digits)):
        if segments['top_left'] in five_digits[j]:
            digits[5] = five_digits[j]
            break
    
    # we can find bottom_left segment by looking at digit 5 and removing middle_segs and top_left
    segments['bottom_right'] = [x for x in digits[5] if x not in segments['middle_segs'] and x != segments['top_left']][0]
    segments['top_right'] = [x for x in segments['right_segs'] if x != segments['bottom_right']][0]
    
    # digit 6 is digit 8 without the top right segment
    digits[6] = ''.join([x for x in digits[8] if x != segments['top_right']])
    
    
    # and we can finnally decode our values after the '|'
    numbers = data[data.index('|')+1:]
    
    # sort every string in the digits to make this easier
    for key in digits:
        digits[key] = ''.join(sorted(digits[key]))

    decoded = ''
    for number in numbers:
        for key in digits:
            if ''.join(sorted(number)) == digits[key]:
                decoded += str(key)
                break
        
    #print("Digits:")
    #for key in digits:
    #    print(key, digits[key])
    #print("\nSegments:")
    #print(segments)
    #
    #print("\nDecoded:", decoded)
    #
    #print("\n\n")
    return int(decoded)
        

total=0
for i in range(len(valores)):
    possibles = valores[i].split(' ')
    #print(possibles)
    total+=get_combination(possibles)
print("The decoded sum of values is:", total)
print("Time:", round((time.perf_counter_ns()-start)/10**6, 3), "ms")

The decoded sum of values is: 1011823
Time: 9.505 ms
